In [63]:
import os
import cv2
import random
import pandas as pd
import tensorflow as tf

In [2]:
EVAL_AMOUNT = 2
DATASET_DIR = "_data/plant_pathology"

In [3]:
train_data = pd.read_csv(os.path.join(DATASET_DIR, "train.csv"))

In [4]:
train_data

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0
...,...,...,...,...,...
1816,Train_1816,0,0,0,1
1817,Train_1817,1,0,0,0
1818,Train_1818,1,0,0,0
1819,Train_1819,0,0,1,0


In [5]:
print(train_data.sum())
print(len(train_data.index))
print(train_data.sum(numeric_only=True).div(len(train_data.index)))
print(train_data.sum(numeric_only=True).sum())

image_id             Train_0Train_1Train_2Train_3Train_4Train_5Trai...
healthy                                                            516
multiple_diseases                                                   91
rust                                                               622
scab                                                               592
dtype: object
1821
healthy              0.283361
multiple_diseases    0.049973
rust                 0.341571
scab                 0.325096
dtype: float64
1821


In [6]:
indices = random.sample(list(train_data.index), k=EVAL_AMOUNT)

In [7]:
def generate_prediction(img):
    pass

In [8]:
print(1063 in train_data.index)

True


In [9]:
import multiprocessing
def execute_in_process(func, args):
    with multiprocessing.get_context("spawn").Pool(1) as pool:
        return pool.apply(func, args)

In [10]:
def get_patches(masks, image, apply_mask=False, padding=0):
    result = []
    
    for mask in masks:
        if apply_mask:
            image_tmp = image * (mask["segmentation"][:, :, np.newaxis])
        else:
            image_tmp = image
        
        bbox = mask["bbox"]
        x0 = bbox[1]-padding
        if x0 < 0:
            x0 = 0
        x1 = bbox[1]+bbox[3]+padding
        if x1 >= image.shape[0]:
            x1 = image.shape[0] - 1
        y0 = bbox[0]-padding
        if y0 < 0:
            y0 = 0
        y1 = bbox[0]+bbox[2]+padding
        if y1 >= image.shape[1]:
            y1 = image.shape[1] - 1
        
        patch = image_tmp[x0:x1, y0:y1]
        
        if 0 in patch.shape:
            continue
        result.append(patch)
    
    return result

In [32]:
device = "cuda"

In [33]:
def sam_generate_mask(image):
    import sys
    sys.path.append("..")
    from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

    sam_checkpoint = "sam_vit_h_4b8939.pth"
    model_type = "vit_h"


    sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
    sam.to(device=device)

    mask_generator = SamAutomaticMaskGenerator(sam)
    masks = mask_generator.generate(image)
    return masks

In [34]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import resnet50, ResNet50_Weights
class BinaryResnetClassifier(nn.Module):
    def __init__(self, num_classes=1):
        super(BinaryResnetClassifier, self).__init__()
        # Load a pre-trained ResNet model
        self.resnet = resnet50(ResNet50_Weights.IMAGENET1K_V1)  # You can choose any ResNet variant
        # Modify the last fully connected layer
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)
        nn.init.xavier_normal_(self.resnet.fc.weight)

    def forward(self, x):
        # Pass the input through the ResNet
        x = self.resnet(x)
        return x
    
import torchvision.transforms.v2 as transforms

tf = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.ToDtype(torch.float32, scale=True),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  
])

def s1_sam_resnet(image):
    #masks = execute_in_process(sam_generate_mask, (image,))
    masks = sam_generate_mask(image)
    patches = get_patches(masks, image)
    resnet = torch.load("../leaf_segmentation/out/leaf_classifier/resnet/resnet_latest.pth")
    resnet = resnet.to(device)
    from PIL import Image
    results = []
    with torch.no_grad():
        for patch in patches:
            input = tf(Image.fromarray(patch)).unsqueeze(0).to(device)
            result = torch.sigmoid(resnet(input)).cpu().item()

            results.append(result)
    del resnet
    PROBABILITY_THRESHOLD = .1
    masks_filtered = [mask for mask, result in zip(masks,results) if result > PROBABILITY_THRESHOLD]
    return masks_filtered

/home/stefan.steinheber/.conda/envs/total/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [23]:
def s1_sam_yolo(image):
    from ultralytics import YOLO, checks
    masks = sam_generate_mask(image)
    patches = get_patches(masks, image)
    model = YOLO("../leaf_segmentation/out/yolo_urban_street/train/weights/best.pt")
    model.info()
    results_yolo = []
    for i, patch in enumerate(patches):
        result = model.predict(patch)
        results_yolo.append(1 if result[0].masks is not None and len(result[0].masks) > 0 else 0)
        masks[i]["patch"] = patch
        masks[i]["leaf_probability"] = 1 if result[0].masks is not None and len(result[0].masks) > 0 else 0
    masks_filtered = [mask for mask, result in zip(masks,results_yolo) if result > 0]
    return masks_filtered

In [24]:
def s1_mask_rcnn(image):
    pass

In [25]:
stage1_dict = {
    "SAM + YOLOv8": s1_sam_yolo,
    "SAM + ResNet": s1_sam_resnet,
    "Mask R-CNN": s1_mask_rcnn
}

In [26]:
import keras

In [27]:
def get_latest_checkpoint(dir):
    if not os.path.isdir(dir):
        return None
    ckpts = [file for file in os.listdir(dir) if file.endswith("keras")]
    ckpts.sort()
    return os.path.join(dir, ckpts[-1])

In [28]:
PROB_THRESHOLD = .8

In [29]:
results_df = pd.DataFrame()

In [30]:
import torch

In [39]:
stage1_results = {}
for stage1_name, stage1_model in stage1_dict.items():
    stage1_results[stage1_name] = []
    for index in indices:
        gt_healthy = bool(train_data.loc[index]["healthy"])
        img = cv2.imread(os.path.join(DATASET_DIR, "images", train_data.loc[index]["image_id"] + ".jpg"))
        with torch.no_grad():
            leaf_masks = stage1_model(img)
            stage1_results[stage1_name].append(leaf_masks)
        torch.cuda.empty_cache()

YOLOv8m-seg summary: 355 layers, 24,605,027 parameters, 0 gradients, 99.2 GFLOPs

0: 480x640 1 leaf, 9.2ms
Speed: 2.2ms preprocess, 9.2ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 448x640 1 leaf, 9.7ms
Speed: 1.6ms preprocess, 9.7ms inference, 1.5ms postprocess per image at shape (1, 3, 448, 640)

0: 640x224 (no detections), 10.3ms
Speed: 1.2ms preprocess, 10.3ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 224)

0: 448x640 1 leaf, 9.2ms
Speed: 1.9ms preprocess, 9.2ms inference, 1.5ms postprocess per image at shape (1, 3, 448, 640)

0: 640x320 1 leaf, 9.4ms
Speed: 1.4ms preprocess, 9.4ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 320)

0: 384x640 1 leaf, 9.3ms
Speed: 1.3ms preprocess, 9.3ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 224x640 1 leaf, 10.1ms
Speed: 1.2ms preprocess, 10.1ms inference, 1.5ms postprocess per image at shape (1, 3, 224, 640)

0: 640x384 1 leaf, 9.7ms
Speed: 1.5ms preprocess, 9

In [40]:
stage1_results

{'SAM + YOLOv8': [[{'segmentation': array([[False, False, False, ..., False, False, False],
           [False, False, False, ..., False, False, False],
           [False, False, False, ..., False, False, False],
           ...,
           [False, False, False, ..., False, False, False],
           [False, False, False, ..., False, False, False],
           [False, False, False, ..., False, False, False]]),
    'area': 1023044,
    'bbox': [416, 110, 1464, 1052],
    'predicted_iou': 1.036436915397644,
    'point_coords': [[1248.0, 703.828125]],
    'stability_score': 0.9924018979072571,
    'crop_box': [0, 0, 2048, 1365],
    'patch': array([[[ 67, 103, 119],
            [ 67, 103, 119],
            [ 67, 103, 119],
            ...,
            [187, 228, 220],
            [187, 229, 218],
            [187, 229, 218]],
    
           [[ 69, 103, 119],
            [ 69, 103, 119],
            [ 69, 103, 119],
            ...,
            [187, 228, 220],
            [187, 229, 218],
  

In [46]:
stage2_names = ["InceptionV3", "VGG19", "MobileNetV3Large"]

In [47]:
stage2_dict = {
    name: keras.models.load_model(get_latest_checkpoint(f"../disease_detection/checkpoints/{name}_rgb")) 
    for name in stage2_names 
    if get_latest_checkpoint(f"../disease_detection/checkpoints/{name}_rgb") is not None
}

I0000 00:00:1730754578.551342 1916614 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38178 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:c1:00.0, compute capability: 8.0


In [48]:
stage2_dict.keys()

dict_keys(['InceptionV3', 'VGG19', 'MobileNetV3Large'])

In [53]:
stage1_results

{'SAM + YOLOv8': [[{'segmentation': array([[False, False, False, ..., False, False, False],
           [False, False, False, ..., False, False, False],
           [False, False, False, ..., False, False, False],
           ...,
           [False, False, False, ..., False, False, False],
           [False, False, False, ..., False, False, False],
           [False, False, False, ..., False, False, False]]),
    'area': 1023044,
    'bbox': [416, 110, 1464, 1052],
    'predicted_iou': 1.036436915397644,
    'point_coords': [[1248.0, 703.828125]],
    'stability_score': 0.9924018979072571,
    'crop_box': [0, 0, 2048, 1365],
    'patch': array([[[ 67, 103, 119],
            [ 67, 103, 119],
            [ 67, 103, 119],
            ...,
            [187, 228, 220],
            [187, 229, 218],
            [187, 229, 218]],
    
           [[ 69, 103, 119],
            [ 69, 103, 119],
            [ 69, 103, 119],
            ...,
            [187, 228, 220],
            [187, 229, 218],
  

In [68]:
stage2_results = {}
for stage1_name, stage1_result in stage1_results.items():
    for leaf_masks in stage1_result:
        for leaf_mask in leaf_masks:
            for stage2_name, stage2_model in stage2_dict.items():
                stage2_results[f"{stage1_name} + {stage2_name}"] = []
                inputs = leaf_mask["patch"]
                inputs = cv2.resize(inputs, (224, 224))
                inputs = tf.constant(inputs)
                inputs = tf.expand_dims(inputs, 0)
                prob = stage2_model(inputs)
                if prob > PROB_THRESHOLD:
                    stage2_results[f"{stage1_name} + {stage2_name}"].append(index)
                    results.append(index)
                    print(f"index {index} sick")

E0000 00:00:1730755566.034403 1916614 cuda_dnn.cc:522] Loaded runtime CuDNN library: 9.1.0 but source was compiled with: 9.3.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
2024-11-04 22:26:06.034563: W tensorflow/core/framework/op_kernel.cc:1841] OP_REQUIRES failed at conv_ops_impl.h:1204 : INVALID_ARGUMENT: No DNN in stream executor.
2024-11-04 22:26:06.034609: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: INVALID_ARGUMENT: No DNN in stream executor.


InvalidArgumentError: Exception encountered when calling Conv2D.call().

[1m{{function_node __wrapped__Conv2D_device_/job:localhost/replica:0/task:0/device:GPU:0}} No DNN in stream executor. [Op:Conv2D][0m

Arguments received by Conv2D.call():
  • inputs=tf.Tensor(shape=(1, 224, 224, 3), dtype=float32)